# Joining Data with Pandas
## 1.0 Data merging basics
### 1.1 inner joins
Inner joins return rows that have matching values in both tables

df1_df2 = df1.merge(df2, on = "common_column", suffixes = ("_df1", "_df2"))


In [25]:
# load libraries 
import pandas as pd

In [26]:
# load the dataFrames
cal = pd.read_pickle("cta_calendar.p")
ridership = pd.read_pickle("cta_ridership.p")
stations = pd.read_pickle("stations.p")

In [27]:
# view the data info
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      365 non-null    int64 
 1   month     365 non-null    int64 
 2   day       365 non-null    int64 
 3   day_type  365 non-null    object
dtypes: int64(3), object(1)
memory usage: 11.5+ KB


In [28]:
ridership.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285 entries, 0 to 3284
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   station_id  3285 non-null   object
 1   year        3285 non-null   int64 
 2   month       3285 non-null   int64 
 3   day         3285 non-null   int64 
 4   rides       3285 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 128.4+ KB


In [29]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   station_id    144 non-null    object
 1   station_name  144 non-null    object
 2   location      144 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB


In [32]:
# merge the three tables
ridership_cal_stations = ridership.merge(cal, on = ["year", "month", "day"]) \
    .merge(stations, on = "station_id")
ridership_cal_stations.head()

,station_id,year,month,day,rides,day_type,station_name,location
0,40010,2019,1,1,576,Sunday/Holiday,Austin-Forest Park,"(41.870851, -87.776812)"
1,40010,2019,1,2,1457,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
2,40010,2019,1,3,1543,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
3,40010,2019,1,4,1621,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
4,40010,2019,1,5,719,Saturday,Austin-Forest Park,"(41.870851, -87.776812)"


In [35]:
# find the total number of rides provided to passengers passing through the Wilson station (station_name == 'Wilson') 
# when riding Chicago's public transportation system on weekdays (day_type == 'Weekday') in July (month == 7). 

# Create a variable called filter_criteria to select the appropriate rows from the merged table 
# so that you can sum the rides column.

# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations["month"] == 7) 
                   & (ridership_cal_stations["day_type"] == "Weekday") 
                   & (ridership_cal_stations["station_name"] == "Wilson"))

In [37]:
# Use .loc and the filter to select for rides.
# Wilson station had 140,005 riders during weekdays in July.
ridership_cal_stations.loc[filter_criteria, 'rides'].sum()

140005

### 1.2 One to many merge with multiple tables 

assume that you are looking to start a business in the city of Chicago. Your perfect idea is to start a company that uses goats to mow the lawn for other businesses. However, you have to choose a location in the city to put your goat farm. You need a location with a great deal of space and relatively few businesses and people around to avoid complaints about the smell. You will need to merge three tables to help you choose your location. The land_use table has info on the percentage of vacant land by city ward. The census table has population by ward, and the licenses table lists businesses by ward.

In [38]:
# load the dataframes
land_use = pd.read_pickle("land_use.p")
licenses = pd.read_pickle("licenses.p")
census = pd.read_pickle("census.p")


In [39]:
# view the dataframe info
land_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ward         50 non-null     object
 1   residential  50 non-null     int64 
 2   commercial   50 non-null     int64 
 3   industrial   50 non-null     int64 
 4   vacant       50 non-null     int64 
 5   other        50 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.5+ KB


In [40]:
# view the dataframe info
licenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   account   10000 non-null  object
 1   ward      10000 non-null  object
 2   aid       4657 non-null   object
 3   business  10000 non-null  object
 4   address   10000 non-null  object
 5   zip       9996 non-null   object
dtypes: object(6)
memory usage: 468.9+ KB


In [41]:
# view the dataframe info
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ward      50 non-null     object
 1   pop_2000  50 non-null     int64 
 2   pop_2010  50 non-null     int64 
 3   change    50 non-null     object
 4   address   50 non-null     object
 5   zip       50 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.5+ KB


In [42]:
# Merge land_use and census and merge result with licenses including suffixes
land_cen_lic = land_use.merge(census,on = "ward") \
                        .merge(licenses, on = "ward", suffixes = ["_cen", "_lic"])
land_cen_lic.head()

,ward,residential,commercial,industrial,vacant,other,pop_2000,pop_2010,change,address_cen,zip_cen,account,aid,business,address_lic,zip_lic
0,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622
1,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14446,743,EMPTY BOTTLE INC,1035 N WESTERN AVE 1ST,60622
2,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14624,775,LITTLE MEL'S HOT DOG,2205 N CALIFORNIA AVE,60647
3,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14987,NaN,MR. BROWN'S LOUNGE,2301 W CHICAGO AVE 1ST,60622
4,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,15642,814,Beat Kitchen,2000-2100 W DIVISION ST,60622


In [43]:
# Group land_cen_lic by ward, pop_2010 (the population in 2010), and vacant, 
# then count the number of accounts.
pop_vac_lic = land_cen_lic.groupby(["ward","pop_2010","vacant"], \
                                   as_index=False).agg({'account':'count'})

pop_vac_lic

,ward,pop_2010,vacant,account
0,1,56149,2,253
1,10,51535,14,130
2,11,51497,5,201
3,12,52235,4,255
4,13,53722,1,101
5,14,54031,3,202
6,15,51501,6,103
7,16,51954,13,156
8,17,51846,5,109
9,18,52992,3,124


In [44]:
# Sort pop_vac_lic by vacant, account, andpop_2010 in descending, ascending, and ascending order respectively.
sorted_pop_vac_lic = pop_vac_lic.sort_values(["vacant", "account", "pop_2010"], 
                                             ascending = [False, True, True])

sorted_pop_vac_lic

,ward,pop_2010,vacant,account
47,7,51581,19,80
12,20,52372,15,123
1,10,51535,14,130
16,24,54909,13,98
7,16,51954,13,156
22,3,53039,13,173
20,28,55199,11,189
46,6,52341,8,149
27,34,51599,7,99
14,22,53515,7,156
